In [135]:
import numpy as np
import scipy as scp
import scipy.stats as ss
from scipy.integrate import quad
from functools import partial
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from datetime import date
now = datetime.today().strftime('%Y:%m:%d')
d1 = datetime.strptime(now, '%Y:%m:%d')
aapl = yf.Ticker("AAPL")
expiry = aapl.options[6]         #expiry date choice
opt = aapl.option_chain(expiry)  #selecting that option based on expiry date                         
low = opt.calls.iloc[15]          #selecting specific call
d2 = datetime.strptime(expiry, '%Y-%m-%d')
diff = (d2 - d1).days
print(opt.calls)

         contractSymbol       lastTradeDate  strike  lastPrice    bid    ask  \
0   AAPL200828C00310000 2020-07-09 18:50:19   310.0      75.28  77.15  80.20   
1   AAPL200828C00320000 2020-07-13 19:23:00   320.0      70.05  68.25  70.85   
2   AAPL200828C00335000 2020-07-13 14:54:38   335.0      66.00  54.80  56.45   
3   AAPL200828C00340000 2020-07-14 14:19:01   340.0      45.38  51.70  53.25   
4   AAPL200828C00345000 2020-07-14 14:56:17   345.0      43.80  46.55  48.20   
5   AAPL200828C00350000 2020-07-10 16:16:40   350.0      40.04  42.95  44.25   
6   AAPL200828C00357500 2020-07-14 18:22:59   357.5      38.85  37.75  39.45   
7   AAPL200828C00360000 2020-07-14 18:34:21   360.0      37.50  36.05  37.40   
8   AAPL200828C00362500 2020-07-13 16:26:02   362.5      31.30  33.65  34.80   
9   AAPL200828C00365000 2020-07-13 18:17:26   365.0      29.35  31.95  34.05   
10  AAPL200828C00367500 2020-07-13 19:24:47   367.5      31.50  30.35  32.50   
11  AAPL200828C00370000 2020-07-14 18:37

In [136]:
S0 = aapl.info['open']
K= low.loc['strike']    # strike
T= diff/365      # maturity 
r= 0.11       # risk free rate 
sig = low.loc['impliedVolatility']     # diffusion coefficient or volatility

In [141]:
N = 5000              # number of periods or number of time steps  
payoff = "call"        # payoff 

dT = float(T) / N                             # Delta t
u = np.exp(sig * np.sqrt(dT))                 # up factor
d = 1.0 / u                                   # down factor 

V = np.zeros(N+1)                             # initialize the price vector
S_T = np.array( [(S0 * u**j * d**(N - j)) for j in range(N + 1)] )  # price S_T at time T

a = np.exp(r * dT)    # risk free compounded return
p = (a - d)/ (u - d)  # risk neutral up probability
q = 1.0 - p           # risk neutral down probability   

if payoff =="call":
    V[:] = np.maximum(S_T-K, 0.0)
else:
    V[:] = np.maximum(K-S_T, 0.0)

for i in range(N-1, -1, -1):
    V[:-1] = np.exp(-r*dT) * (p * V[1:] + q * V[:-1])    # the price vector is overwritten at each step
        
print("BS Tree Price: ", V[0])

BS Tree Price:  22.792644100356572


In [142]:
print("Difference between model and actual: ", (low.loc['lastPrice'] - V[0]))

Difference between model and actual:  1.5073558996434286
